In [1]:
import pandas as pd
import numpy as np
import librosa

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import (
    Dense,
    Conv1D,
    MaxPooling1D,
    Flatten,
    Dropout,
    BatchNormalization,
)
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

### Read dataset

In [ ]:
train_dataset = pd.read_excel("./datasets/train_dataset.xlsx")
test_dataset = pd.read_excel("./datasets/test_dataset.xlsx")

### Data Augmentation

In [3]:
def noise(data):
    """
    Add random Gaussian noise to the audio signal for data augmentation.

    This function adds controlled random noise to the input audio signal. The amplitude
    of the noise is proportional to the maximum amplitude of the input signal,
    making it adaptive to different audio volumes.

    Parameters
    ----------
    data : numpy.ndarray
        Input audio signal time series

    Returns
    -------
    numpy.ndarray
        Audio signal with added noise, same shape as input

    Notes
    -----
    The noise generation process:
    1. Calculate noise amplitude as 3.5% of input signal's max amplitude
    2. Generate Gaussian noise with the same length as input
    3. Scale noise by calculated amplitude
    4. Add scaled noise to original signal

    The noise amplitude is randomized using uniform distribution to create
    variety in the augmented data.
    """
    noise_amp = 0.035 * np.random.uniform() * np.amax(data)
    data = data + noise_amp * np.random.normal(size=data.shape[0])
    return data


def stretch(data, rate=0.8):
    """
    Time-stretch the audio signal without changing its pitch.

    Parameters
    ----------
    data : numpy.ndarray
        Input audio signal
    rate : float, optional
        Stretching rate. Values > 1 speed up the audio, values < 1 slow it down.
        Default is 0.8 (20% slower).

    Returns
    -------
    numpy.ndarray
        Time-stretched audio signal
    """
    return librosa.effects.time_stretch(data, rate=rate)


def shift(data):
    """
    Randomly shift the audio signal in time.

    Parameters
    ----------
    data : numpy.ndarray
        Input audio signal

    Returns
    -------
    numpy.ndarray
        Time-shifted audio signal, shifted by -5000 to 5000 samples
    """
    shift_range = int(np.random.uniform(low=-5, high=5) * 1000)
    return np.roll(data, shift_range)


def pitch(data, sampling_rate, pitch_factor=0.7):
    """
    Shift the pitch of the audio signal.

    Parameters
    ----------
    data : numpy.ndarray
        Input audio signal
    sampling_rate : int
        Sampling rate of the input audio
    pitch_factor : float, optional
        Number of semitones to shift. Default is 0.7 (lower pitch)

    Returns
    -------
    numpy.ndarray
        Pitch-shifted audio signal
    """
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=pitch_factor)

We use only noise and stretch, copying the steps from kaggle notebook

In [4]:
def extract_audio_features(data, sample_rate):
    """
    Extract audio features from the input audio data for emotion recognition.

    This function extracts multiple audio features that are useful for speech emotion recognition:
    - Zero Crossing Rate (ZCR): Rate at which the signal changes from positive to negative
    - Chroma STFT: Represents the spectral energy across the 12 pitch classes
    - MFCC (Mel-frequency cepstral coefficients): Represents the short-term power spectrum
    - RMS (Root Mean Square): Represents the loudness of the signal
    - Mel Spectrogram: Represents the power spectral density on mel-scale

    Parameters
    ----------
    data : numpy.ndarray
        Audio time series data loaded using librosa

    Returns
    -------
    numpy.ndarray
        1D array containing concatenated features in the following order:
        [ZCR, Chroma STFT, MFCC, RMS, Mel Spectrogram]

    Notes
    -----
    All features are averaged across time using mean to get a fixed-length
    representation regardless of the input audio length.
    """
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result = np.hstack((result, zcr))  # stacking horizontally

    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft))

    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc))

    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms))

    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel))

    return result


def augment_and_get_features(path):
    """
    Load an audio file, apply data augmentation, and extract features.

    This function performs the following steps:
    1. Loads the audio file
    2. Extracts features from the original audio
    3. Applies noise augmentation and extracts features
    4. Applies time stretching followed by pitch shifting and extracts features

    Parameters
    ----------
    path : str
        Path to the audio file

    Returns
    -------
    numpy.ndarray
        2D array of shape (3, n_features) containing features from:
        - Row 0: Original audio
        - Row 1: Noise augmented audio
        - Row 2: Stretch and pitch augmented audio
    """
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path)

    # without augmentation
    res1 = extract_audio_features(data, sample_rate)
    result = np.array(res1)

    # data with noise
    noise_data = noise(data)
    res2 = extract_audio_features(noise_data, sample_rate)
    result = np.vstack((result, res2))  # stacking vertically

    # data with stretching and pitching
    new_data = stretch(data)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = extract_audio_features(data_stretch_pitch, sample_rate)
    result = np.vstack((result, res3))  # stacking vertically

    return result


def get_features(path):
    """
    Load an audio file, apply data augmentation, and extract features.

    This function performs the following steps:
    1. Loads the audio file
    2. Extracts features from the original audio

    Parameters
    ----------
    path : str
        Path to the audio file

    Returns
    -------
    numpy.ndarray
        2D array of shape (1, n_features) containing features from:
        - Row 0: Original audio
    """
    data, sample_rate = librosa.load(path)

    result = extract_audio_features(data, sample_rate)
    result = np.array(result)

    return result

In [ ]:
x_train = train_dataset["Path"]
y_train = train_dataset["Emotion"]
x_test = test_dataset["Path"]
y_test = test_dataset["Emotion"]

In [ ]:
x_train_augmented, y_train_augmented = [], []
for path, emotion in zip(x_train, y_train):
    feature = augment_and_get_features(path)
    for ele in feature:
        x_train_augmented.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        y_train_augmented.append(emotion)

In [ ]:
len(x_train_augmented), len(y_train_augmented)

In [ ]:
type(x_train_augmented), type(y_train_augmented)

no. of features

In [ ]:
len(x_train_augmented[0])

In [ ]:
train_features = pd.DataFrame(x_train_augmented)
train_features["Emotion"] = y_train_augmented
train_features.head(2)

Since, augmenting and extracting features from audio clips is a time taking process, `train_features` data can be saved in temporary directory by uncommenting the below code, to make it easier to continue later

In [ ]:
train_augmented_path = "./datasets/temp_data/train_augmented.xlsx"
# train_features.to_excel("./datasets/temp_data/train_augmented.xlsx", index=False)
# train_features = pd.read_excel(train_augmented_path)

### Data Preparation

In [ ]:
x_test, y_test = [], []
for path, emotion in zip(test_dataset.Path, test_dataset.Emotion):
    features = get_features(path)
    x_test.append(features)
    y_test.append(emotion)

In [ ]:
x_test, y_test

In [ ]:
test_features = pd.DataFrame(x_test)
test_features["Emotion"] = y_test
test_features.head(2)

Uncomment below code to save `test_features` data in temporary directory to continue later

In [ ]:
test_features_path = "./datasets/temp_data/test_features.xlsx"
# test_features.to_excel(test_features_path, index=False)
# test_features = pd.read_excel(test_features_path)

In [18]:
y_train = train_features[["Emotion"]].copy()
y_train.columns = ["Emotion"]
x_train = train_features.drop("Emotion", axis=1)
x_train.shape, y_train.shape

((27363, 162), (27363, 1))

In [19]:
y_test = test_features[["Emotion"]].copy()
y_test.columns = ["Emotion"]
x_test = test_features.drop("Emotion", axis=1)
x_test.shape, y_test.shape

((3041, 162), (3041, 1))

One Hot Encoding

In [20]:
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()

In [21]:
y_train, encoder.categories_

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]]),
 [array(['angry', 'calm', 'disgust', 'fear', 'happy', 'neutral', 'sad',
         'surprise'], dtype=object)])

Scaler

In [22]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
x_train, x_test

(array([[-0.70397485,  0.06949796,  0.21696521, ..., -0.28704386,
         -0.27498211, -0.2201443 ],
        [ 1.40917882,  1.32117643,  1.58257157, ..., -0.10925702,
         -0.08990944, -0.00775504],
        [-0.35930966,  0.35376681, -0.20417907, ..., -0.28704387,
         -0.27498213, -0.22014433],
        ...,
        [ 0.48491295,  0.45261468, -0.03170085, ..., -0.28704244,
         -0.27498062, -0.22014267],
        [ 2.06113142,  1.20598425,  1.32075349, ..., -0.09408117,
         -0.06059889,  0.00321045],
        [ 0.67179553, -0.25184093, -0.11018163, ..., -0.28704261,
         -0.27498109, -0.22014387]]),
 array([[ 0.05039082, -0.34379398, -0.29185215, ..., -0.25565859,
         -0.25299771, -0.25053177],
        [ 0.17937001,  0.68275129,  0.81617919, ..., -0.25565855,
         -0.25299767, -0.25053129],
        [-0.49825688,  1.19695172,  0.86664828, ..., -0.25565854,
         -0.25299765, -0.2505311 ],
        ...,
        [-0.70697849, -0.09515453, -0.28566544, ..., -

Expand dimensions, because model expects a specific shape of data (3D)

In [23]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, x_test.shape

((27363, 162, 1), (3041, 162, 1))

### Modelling

In [24]:
model = Sequential()
model.add(
    Conv1D(
        256,
        kernel_size=5,
        strides=1,
        padding="same",
        activation="relu",
        input_shape=(x_train.shape[1], 1),
    )
)
model.add(MaxPooling1D(pool_size=5, strides=2, padding="same"))

model.add(Conv1D(256, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=5, strides=2, padding="same"))

model.add(Conv1D(128, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=5, strides=2, padding="same"))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=5, strides=2, padding="same"))

model.add(Flatten())
model.add(Dense(units=32, activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(units=8, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

d:\ml\SER_training\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 162, 256)            │           1,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 81, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 81, 256)             │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 41, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 41, 128)             │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 21, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 21, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 21, 64)              │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 11, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 704)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │          22,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │             264 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 557,288 (2.13 MB)

 Trainable params: 557,288 (2.13 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
rlrp = ReduceLROnPlateau(
    monitor="loss", factor=0.4, verbose=0, patience=2, min_lr=10e-8
)
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=25,
    validation_data=(x_test, y_test),
    callbacks=[rlrp],
)

Epoch 1/25
428/428 ━━━━━━━━━━━━━━━━━━━━ 24s 41ms/step - accuracy: 0.2529 - loss: 1.8263 - val_accuracy: 0.3992 - val_loss: 1.4964 - learning_rate: 0.0010
Epoch 2/25
428/428 ━━━━━━━━━━━━━━━━━━━━ 17s 40ms/step - accuracy: 0.3869 - loss: 1.4993 - val_accuracy: 0.4331 - val_loss: 1.3815 - learning_rate: 0.0010
Epoch 3/25
428/428 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.4251 - loss: 1.4039 - val_accuracy: 0.4594 - val_loss: 1.3432 - learning_rate: 0.0010
Epoch 4/25
428/428 ━━━━━━━━━━━━━━━━━━━━ 17s 40ms/step - accuracy: 0.4536 - loss: 1.3407 - val_accuracy: 0.4765 - val_loss: 1.3035 - learning_rate: 0.0010
Epoch 5/25
428/428 ━━━━━━━━━━━━━━━━━━━━ 17s 41ms/step - accuracy: 0.4694 - loss: 1.3020 - val_accuracy: 0.5192 - val_loss: 1.2180 - learning_rate: 0.0010
Epoch 6/25
428/428 ━━━━━━━━━━━━━━━━━━━━ 18s 42ms/step - accuracy: 0.4757 - loss: 1.2686 - val_accuracy: 0.5248 - val_loss: 1.1991 - learning_rate: 0.0010
Epoch 7/25
428/428 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.4856 - l

In [26]:
type(model)

keras.src.models.sequential.Sequential

### Save model

In [27]:
model_save_path = "./saved_models/model.keras"
model.save(model_save_path)

In [28]:
model.evaluate(x_test, y_test)

96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5522 - loss: 1.2399


[1.245154619216919, 0.5412693023681641]